# Hourly temperature averages in Qatar

#### Load python tools

In [1]:
%load_ext lab_black

In [5]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles

In [6]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [70]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

## Get World Cup matches by date/time

In [100]:
sked_df = pd.read_csv(
    "data/processed/world_cup_matches_schedule.csv",
    parse_dates=["local_date", "pst_date"],
)

In [103]:
sked_df["time"] = pd.to_datetime(sked_df["local_time"]).dt.strftime("%H:%M")
sked_df["date"] = pd.to_datetime(sked_df["local_date"]).dt.strftime("%m-%d")

## Get weather averages from [weatherspark](https://weatherspark.com/s/105083/2/Average-Fall-Weather-in-Doha-Qatar#Figures-Temperature)

In [71]:
src = pd.read_csv(
    "data/raw/weather/qatar/averages_hamad_airport/Hourly Averages for 149641 Hamad International Airport.csv"
)

In [72]:
src.columns = (
    src.columns.str.lower()
    .str.strip()
    .str.replace("/", "_", regex=False)
    .str.replace(" (°f)", "", regex=False)
    .str.replace(" (%)", "_pct", regex=False)
    .str.replace(" (in)", "_inch", regex=False)
    .str.replace(" (hr)", "_hour", regex=False)
    .str.replace(" (mph)", "_mph", regex=False)
    .str.strip()
)

In [73]:
src["time"] = pd.to_datetime(src["timestamp"]).dt.strftime("%H:%M")
src["date"] = pd.to_datetime(src["timestamp"]).dt.strftime("%m-%d")

#### Just the temperature columns

In [74]:
temp_cols = [col for col in src.columns if "temperature_" in col]
pct_cols = [col for col in src.columns if "_pct" in col]

In [75]:
src_slim = src[
    [
        "date",
        "time",
        "temperature_mean",
        "temperature_percentile10th",
        "temperature_percentile25th",
        "temperature_percentile75th",
        "temperature_percentile90th",
        "temperature_frigid_pct",
        "temperature_freezing_pct",
        "temperature_chilly_pct",
        "temperature_cold_pct",
        "temperature_cool_pct",
        "temperature_comfortable_pct",
        "temperature_warm_pct",
        "temperature_hot_pct",
        "temperature_sweltering_pct",
    ]
].copy()

In [76]:
src_melt = pd.melt(
    src_slim,
    id_vars=["date", "time"],
    value_vars=[
        "temperature_mean",
        "temperature_percentile10th",
        "temperature_percentile25th",
        "temperature_percentile75th",
        "temperature_percentile90th",
        "temperature_frigid_pct",
        "temperature_freezing_pct",
        "temperature_chilly_pct",
        "temperature_cold_pct",
        "temperature_cool_pct",
        "temperature_comfortable_pct",
        "temperature_warm_pct",
        "temperature_hot_pct",
        "temperature_sweltering_pct",
    ],
    var_name="category",
    value_name="value",
)

In [77]:
category_list = [
    "temperature_frigid_pct",
    "temperature_freezing_pct",
    "temperature_chilly_pct",
    "temperature_cold_pct",
    "temperature_cool_pct",
    "temperature_comfortable_pct",
    "temperature_warm_pct",
    "temperature_hot_pct",
    "temperature_sweltering_pct",
]

In [78]:
category_df = src_melt[src_melt["category"] == "temperature_mean"]

In [79]:
len(category_df)

In [86]:
category_df

,date,time,category,value
0,01-01,00:00,temperature_mean,62.49
1,01-01,01:00,temperature_mean,62.49
2,01-01,02:00,temperature_mean,61.95
3,01-01,03:00,temperature_mean,61.45
4,01-01,04:00,temperature_mean,60.97
...,...,...,...,...
8755,12-31,19:00,temperature_mean,66.45
8756,12-31,20:00,temperature_mean,65.69
8757,12-31,21:00,temperature_mean,64.99
8758,12-31,22:00,temperature_mean,64.48


In [105]:
sked_df.head(1)

,idmatch,matchnumber,group_name,home_team,away_team,local_date,local_time,pst_date,pst_time,time,date
0,400128082,1,Group A,QAT,ECU,2022-11-21,16:00:00,2022-11-21,08:00:00,16:00,11-21


In [147]:
matches = (
    alt.Chart(sked_df).mark_circle(size=3, color="black").encode(x="date:O", y="time:O")
)
matches.properties(width=800)

alt.Chart(...)

In [148]:
category_df.head()

,date,time,category,value
0,01-01,00:00,temperature_mean,62.49
1,01-01,01:00,temperature_mean,62.49
2,01-01,02:00,temperature_mean,61.95
3,01-01,03:00,temperature_mean,61.45
4,01-01,04:00,temperature_mean,60.97


In [204]:
heatmap = (
    alt.Chart(category_df)
    .mark_rect()
    .encode(
        x=alt.X(
            "date:O",
            axis=alt.Axis(
                values=["01-01", "04-01", "07-01", "10-01", "12-01"], title=" "
            ),
        ),
        y=alt.Y(
            "time:O",
            axis=alt.Axis(
                values=["00:00", "06:00", "12:00", "18:00", "23:00"], title=" "
            ),
        ),
        color=alt.Color(
            "value:Q",
            title="Avg. hourly temp",
            scale=alt.Scale(scheme="spectral", reverse=True),
        ),
    )
)
heatmap.properties(width=800)

alt.Chart(...)

In [205]:
dates_df = pd.DataFrame(
    [{"x_min": "11-21", "x_max": "12-18", "y_min": "00:00", "y_max": "23:00"}]
)

In [206]:
dates = (
    alt.Chart(dates_df)
    .mark_rect(color="#e9e9e9", opacity=0.3)
    .encode(
        x=alt.X("x_min:O", title=" "),
        x2="x_max:O",
        y=alt.Y("y_min", title=" "),
        y2="y_max",
    )
)

In [209]:
(
    heatmap.properties(width=400, height=500)
    + matches.properties(width=400, height=500)
    + dates.properties(width=400, height=500)
).configure_legend(orient="top")

alt.LayerChart(...)